In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
pip install wget

In [0]:
cd/content/gdrive/My Drive/ML

In [0]:
import wget

In [0]:
link = "https://storage.googleapis.com/kaggle-data-sets/59760/840806/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589443141&Signature=ZGP%2FHKK0%2BdO%2BpnxLwfnhf%2FEAxMBGjBt1oixwBCi%2FFrP2tBQ6IuKkLZmPuJBf%2BgjynGFWiGtHEtDj5FH4%2B6wuhamJAVQbkmiN2%2BjaKqU4eCHyQT2OVwAF98d2UbDXXX482KkDZ0l%2B2qXONPGT3XM0%2B8sDiFIE9o3n7%2BV%2BX6rAw4Kn%2BYc75CU09u%2Faz5D9GxE65SQfVsLW4UJh7D9Cc5rUTZAw9HRcNNy%2FeqyVlbcE0aXWj8B5pPjEWL%2F%2F2lotzpi0P5BOVPpArQvX30Qp5SeQYyEYrklld5y6rEylGhR9zdnNDz3NQikXLCCv1A%2Bv2yBS%2BGWsCY4VSWyLMlqyNT3Tow%3D%3D&response-content-disposition=attachment%3B+filename%3Danimals10.zip"

In [0]:
wget.download(link)

In [0]:
data = "./"

In [0]:
!unzip -q animals10.zip -d $data

In [0]:
pip install split-folders

In [0]:
import split_folders as sf

In [0]:
sf.ratio("raw-img", output = "./", ratio = (.8,.1,.1))

Copying files: 26179 files [06:14, 69.99 files/s]


In [0]:
train_data = data + "./train"
valid_data = data + "./val"

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(16, (3,3), activation="relu", input_shape = (100,100,3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))

In [0]:
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPool2D(2,2))

In [0]:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation= "softmax"))
model.summary()

In [0]:
gen_train = ImageDataGenerator(rescale=1/255)
gen_valid = ImageDataGenerator(rescale=1/255)

In [0]:
gen_train_data = gen_train.flow_from_directory(
    train_data,
    target_size = (100,100),
    color_mode = "rgb",
    batch_size = 130,
    class_mode = "categorical"
)

In [0]:
gen_valid_data = gen_valid.flow_from_directory(
    valid_data,
    target_size = (100,100),
    color_mode = "rgb",
    batch_size = 130,
    class_mode = "categorical"
)

In [0]:
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics=["acc"])

In [0]:
history = model.fit_generator(
    gen_train_data,
    steps_per_epoch = 1000,
    epochs = 40,
    verbose = 1,
    validation_data = gen_valid_data,
    validation_steps = 1000
)

In [0]:
model.save("classificationAnimals.h5")

In [0]:
import tensorflow as tf
model = tf.keras.models.load_model("classificationAnimals.h5")

In [0]:
className = ["dog", "horse","elephant", "butterfly",  "chicken",  "cat", "cow",  "sheep", "squirrel","spider"]

In [0]:
from google.colab import files
from keras.preprocessing import image

In [0]:
uploaded = files.upload()
for k in uploaded.keys():
    path = "./" + k
    img = image.load_img(path, target_size = (100,100))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis = 0)
    images = np.vstack([x])
    result = model.predict(images)
    predict_label = np.argmax(result)
    plt.imshow(img)
    print("Name is ",className[predict_label])